In [2]:
# imports and set up logging
import gensim 
import logging
import glob, os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [7]:
# directory containing all source texts for training the model 
data_dir="/Users/oliviafeng/Desktop/uchi/digital_text2/rick_and_morty_episodes_txt"

In [8]:
#data_dir="corpus"
os.chdir(data_dir)
documents = list()
for filename in glob.glob("*.txt"):
    filedata = open(filename, 'r').read()
    print(filename + " = " + str(len(filedata)) + " chars")
    documents = documents + filedata.split(".")

episode_4.txt = 25112 chars
episode_5.txt = 23781 chars
episode_7.txt = 26417 chars
episode_6.txt = 21026 chars
episode_2.txt = 32767 chars
episode_3.txt = 21536 chars
episode_1.txt = 26824 chars
episode_0.txt = 30370 chars
episode_10.txt = 32767 chars
episode_11.txt = 29633 chars
episode_13.txt = 644 chars
episode_12.txt = 30026 chars
episode_16.txt = 21591 chars
episode_17.txt = 20546 chars
episode_15.txt = 30903 chars
episode_14.txt = 25946 chars
episode_28.txt = 17601 chars
episode_25.txt = 19829 chars
episode_19.txt = 25438 chars
episode_18.txt = 29884 chars
episode_24.txt = 32767 chars
episode_26.txt = 32767 chars
episode_27.txt = 32767 chars
episode_23.txt = 31211 chars
episode_22.txt = 32767 chars
episode_20.txt = 12670 chars
episode_21.txt = 26546 chars
episode_8.txt = 27402 chars
episode_9.txt = 1480 chars


In [9]:
# Check to see that the first sentence is correct
print(documents[0])

docs2 = []
# remove all the "\n"s
for doc in documents:
    newdoc = doc.replace("\n", " ")
    docs2.append(newdoc)

# one can also do text preprocessing here: lowercase, lemmatize, remove stopwords & punctuation etc

print("----")
print(docs2[0])


(The sky is purple and tumultuous as the sounds of Rick and Morty running and panting are heard
----
(The sky is purple and tumultuous as the sounds of Rick and Morty running and panting are heard


In [34]:
# data preprocessing
import spacy
import re

nlp = spacy.load("en_core_web_sm")

with open("/Users/oliviafeng/Desktop/uchi/digital_text2/word2vec_own_work/stop_words.txt", "r", encoding="utf-8") as f:
    custom_stopwords = set(line.strip().lower() for line in f if line.strip())

cleaned_sentences = []

for doc in docs2:
    spacy_doc = nlp(doc)
    tokens =[]
    for token in spacy_doc:
        lemma = token.lemma_.lower()
        if (
            lemma in custom_stopwords
            or token.is_punct
            or not lemma.isalpha()
            or len(lemma) <= 1  
        ):
            continue

   
        lemma = re.sub(r":$", "", lemma)

        tokens.append(lemma)

    cleaned_sentences.append(tokens)

In [35]:
from gensim.models import word2vec, Phrases
from gensim.models.phrases import Phraser

#documents = ["the mayor of new york was there", "human computer interaction and machine learning has now become a trending research area","human computer interaction is interesting","human computer interaction is a pretty interesting subject", "human computer interaction is a great and new subject", "machine learning can be useful sometimes","new york mayor was present", "I love machine learning because it is a new subject area", "human computer interaction helps people to get user friendly applications"]

# sentence_stream = [doc.split(" ") for doc in docs2]  #documents

trigram_sentences_project = []

bigram = Phraser(Phrases(cleaned_sentences))
trigram = Phraser(Phrases(bigram[cleaned_sentences]))

for sent in cleaned_sentences:
    bigrams_ = bigram[sent]
    trigrams_ = trigram[bigram[sent]]
    trigram_sentences_project.append(trigrams_)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 1    # Minimum word count                        
num_workers = 20      # Number of threads to run in parallel
context = 8           # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
skip_grams = 1        # 0 for CBOW, 1 for skip-gramsword2vec.Word2Vec

model = word2vec.Word2Vec(trigram_sentences_project, workers=num_workers, \
            vector_size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, sg = skip_grams, epochs=10)

# , epochs=10

vocab = list(model.wv.index_to_key) #vocab.keys()
print(vocab[:10])


2025-04-14 23:40:14,147 : INFO : collecting all words and their counts
2025-04-14 23:40:14,148 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2025-04-14 23:40:14,212 : INFO : PROGRESS: at sentence #10000, processed 61264 words and 44294 word types
2025-04-14 23:40:14,222 : INFO : collected 49740 token types (unigram + bigrams) from a corpus of 69853 words and 11231 sentences
2025-04-14 23:40:14,222 : INFO : merged Phrases<49740 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2025-04-14 23:40:14,223 : INFO : Phrases lifecycle event {'msg': 'built Phrases<49740 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000> in 0.08s', 'datetime': '2025-04-14T23:40:14.223309', 'gensim': '4.3.1', 'python': '3.10.17 | packaged by conda-forge | (main, Apr 10 2025, 22:15:49) [Clang 18.1.8 ]', 'platform': 'macOS-14.6.1-x86_64-i386-64bit', 'event': 'created'}
2025-04-14 23:40:14,223 : INFO : exporting phrases from Phrases<49740 vocab, min_count=5, threshold=10.0

['rick', 'morty', 'jerry', 'get', 'summer', 'know', 'beth', 'oh', 'like', 'right']


In [36]:
# Print the total number of items in our model's vocabulary
print(len(model.wv))

7293


In [70]:
w1 = "portal"
w2 = "morty"
w3 = "rick"
w4 = "pickle"
w5 = "fuck"
# model.wv.most_similar (positive=w1)
# model.wv.most_similar (positive=w2)
# model.wv.most_similar (positive=w3)
# model.wv.most_similar (positive=w4)
model.wv.most_similar (positive=w5)

[('bitch', 0.9737847447395325),
 ('ah', 0.9625250101089478),
 ('motherfucker', 0.961152195930481),
 ('shut', 0.9549633264541626),
 ('whoa_whoa', 0.9543323516845703),
 ('monster', 0.9541509747505188),
 ('aah', 0.9520564079284668),
 ('whoa', 0.9516603350639343),
 ('feel_good', 0.951093316078186),
 ('fucking', 0.9504562616348267)]

In [51]:
# Check the "most similar words", using the default "cosine similarity" measure.

result = model.wv.most_similar(positive=["morty", "summer"], negative=['rick'])

most_similar_key, similarity = result[0]  # look at the first match

print(f"{most_similar_key}: {similarity:.4f}")


ethan: 0.8530


In [60]:
positive = ["morty", "summer"]
negative = ["rick"]

# cosine_similarity method
cosine_similarity = model.wv.most_similar(positive=positive, negative=negative)
print(f"the cosine_similarity is:{cosine_similarity}")

# cosmul method
cosmul_similarity = model.wv.most_similar_cosmul(positive=positive, negative=negative)
print(f"the cosmul_similarity is:{cosmul_similarity}")


the cosine_similarity is:[('ethan', 0.8529946804046631), ('cry', 0.8318160176277161), ('roars', 0.8191099166870117), ('aah', 0.8184424638748169), ('gasp', 0.8167917728424072), ('god', 0.8167688250541687), ('sorry', 0.8124842643737793), ('bye', 0.8113460540771484), ('kiss', 0.8111793994903564), ('ugh', 0.8102681636810303)]
the cosmul_similarity is:[('ethan', 0.9562269449234009), ('cry', 0.9454898238182068), ('roars', 0.938749372959137), ('aah', 0.9385322332382202), ('gasp', 0.937446117401123), ('god', 0.9368084669113159), ('sorry', 0.9352295994758606), ('bye', 0.9345331192016602), ('kiss', 0.9342777729034424), ('ugh', 0.9341124892234802)]


In [63]:
model.predict_output_word(["I","love","life"], topn=30)  # also the most basic way one could implement text completion

[('love', 0.0012793476),
 ('morty_jr', 0.0011305797),
 ('life', 0.0010671734),
 ('would', 0.0009596353),
 ('tell', 0.00090086955),
 ('even', 0.0008906037),
 ('planet', 0.00082504586),
 ('father', 0.00081498094),
 ('mean', 0.00075664005),
 ('kind', 0.0007543548),
 ('could', 0.00075212045),
 ('bad', 0.00070563925),
 ('much', 0.0006552428),
 ('need', 0.0006460995),
 ('maybe', 0.00064282544),
 ('whole', 0.00063223013),
 ('son', 0.0006279958),
 ('earth', 0.0006078488),
 ('dada', 0.00060304324),
 ('something', 0.00059853896),
 ('world', 0.0005837073),
 ('gazorpian', 0.00058287894),
 ('die', 0.0005733098),
 ('ever', 0.0005733026),
 ('real', 0.0005696367),
 ('family', 0.0005617296),
 ('want', 0.00054931734),
 ('horse', 0.0005467827),
 ('matter', 0.00053397886),
 ('conquer', 0.00053298543)]

In [64]:
#change these locations to somewhere on your machine
tensorsfp = "/Users/oliviafeng/Desktop/uchi/digital_text2/word2vec_own_work/tensorsfp.txt"
metadatafp = "/Users/oliviafeng/Desktop/uchi/digital_text2/word2vec_own_work/metadatafp.txt"

with open( tensorsfp, 'w+') as tensors:
    with open( metadatafp, 'w+') as metadata:
         for word in model.wv.index_to_key:   #index2word
                metadata.write(word + '\n')
                vector_row = '\t'.join(map(str, model.wv[word]))  #model[word]
                tensors.write(vector_row + '\n')